In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#NER
각 단어가 어떠한 개체명(장소, 시간, 등)을 가르키는지를 확인하는 task  
dataset:https://www.kaggle.com/datasets/debasisdotcom/name-entity-recognition-ner-dataset

##Data PreProcessing  
제공된 data는 이미 tokenization이 되어있는 형태로 제공된다.  
그렇기에 vocab dictionary를 직접 구성하고 각 tokenization된 token들을 indexing하여 숫자형태의 데이터로 변환환다.  
이때 class에 index를 부여해주는 scikit learn의 LabelEncoder를 사용해서 변환환다.  
또한 모든 token의 수를일정하게 맞춰주어야만 tensor 연산이 가능하기 때문에 <PAD>에 대한 indexing도 추가한다.


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch

data_df = pd.read_csv("/content/drive/MyDrive/nlp-open-tutorial/5일차_배포/dataset/NER dataset.csv", encoding="ISO-8859-1").loc[:100000,:]
#print(data_df)

#PAD를 추가하여 token에 index를 부여하기 위한 label encoder 학습
token_encoder = LabelEncoder()
token_encoder.fit(["<PAD>"]+list(map(lambda x: x.lower(),data_df.loc[:,"Word"])))
token_pad_index = token_encoder.transform(["<PAD>"])
#print(token_encoder.transform(["<PAD>"]))

#PAD를 추가하여  label에 index를 부여하기 위한  label encoder 학습
label_encoder = LabelEncoder()
label_encoder.fit(["<PAD>"]+list(data_df.loc[:,"Tag"]))
label_pad_index = label_encoder.transform(["<PAD>"])
#print(label_encoder.transform(["<PAD>"]))

data_list = []
label_list = []
for i in range(len(data_df)):
  if type(data_df.loc[i,"Sentence #"]) is str:
    data = []
    label = []
  data.append(data_df.loc[i,"Word"].lower())
  label.append(data_df.loc[i,"Tag"])
  if type(data_df.loc[i,"Sentence #"]) is str:
    data_list.append(data)
    label_list.append(label)

data_list = list(map(token_encoder.transform, data_list))
label_list = list(map(label_encoder.transform, label_list))

print(len(data_list))
print(len(label_list))

4544
4544


##Dataset  
전체 데이터중 4000개의 data는 train 나머지는 test로 사용한다.  
사전에 tokenization과 indexing된 데이터의 크기를 맞추기위해 data에 padding을 추가한다.  
  padding의 value는 <PAD> token의 index를 사용한다.

In [3]:
import torch
from torch.utils.data import DataLoader
import numpy as np

#train_test_split
train_data_list = data_list[:4000]
test_data_list = data_list[4000:]
train_label_list = label_list[:4000]
test_label_list = label_list[4000:]

class myDataset(Dataset):
  def __init__(self,data_list, label_list) -> None:
      super().__init__()
      self.data_list = data_list
      self.label_list = label_list

  def __len__(self):
      return len(self.label_list)

  def __getitem__(self, index):

      data = torch.tensor(self.data_list[index])
      label = torch.tensor(self.label_list[index])

      #make data with padding
      # data_pad = torch.tensor([token_pad_index] * (50 - len(data))).reshape(-1)
      # label_pad = torch.tensor([label_pad_index] * (50 - len(label))).reshape(-1)
      data_pad = torch.zeros(100,dtype=torch.int64) + token_pad_index
      label_pad = torch.zeros(100)+label_pad_index

      # data = torch.LongTensor(np.concatenate([self.data_list[index], data_pad]))
      # label = torch.LongTensor(np.concatenate([self.label_list[index], label_pad]))
      data = torch.cat([torch.IntTensor(self.data_list[index]),data_pad])[:50]
      label = torch.cat([torch.IntTensor(self.label_list[index]),label_pad])[:50]

      return data.flip(-1), label.flip(-1)

train_dataset = myDataset(train_data_list, train_label_list)
test_dataset = myDataset(test_data_list, test_label_list)

for i in train_dataset:
  print(i)
  print(i[0].shape, i[1].shape)
  break

batch_size = 50
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)

for i in train_dataloader:
  print(i[0].shape)
  print(i[1].shape)
  break

(tensor([ 487,  487,  487,  487,  487,  487,  487,  487,  487,  487,  487,  487,
         487,  487,  487,  487,  487,  487,  487,  487,  487,  487,  487,  487,
         487,  487,   15, 2530, 9163, 3983, 9403, 1687, 6432, 9973, 9165, 2803,
         933, 4970, 4744, 9813, 9165, 7228, 9254, 5570, 9217, 5741, 4409, 2824,
        6432, 9200]), tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 17., 17.,
        17., 17., 17.,  4., 17., 17., 17., 17., 17.,  3., 17., 17., 17., 17.,
        17.,  3., 17., 17., 17., 17., 17., 17.], dtype=torch.float64))
torch.Size([50]) torch.Size([50])
torch.Size([50, 50])
torch.Size([50, 50])


##Model  
lstm을 위한 model과 seq2seq를 이용한 모델 두가지 구성  
seq2seq는 encoder model과 decoder model을 따로 구성하고 encoder model의 h,c를 decoder model의 입력으로 사용한다.  
이후 decoder model의 hidden state 들을 예측을 위해 사용한다.

# LSTM 만들기

In [4]:
from torch import nn

class myModel(nn.Module):
  def __init__(self) -> None:
      super().__init__()
      self.emb = nn.Embedding(50000, 128)
      self.lstm = nn.LSTM(128, 128)
      self.ln1 = nn.Linear(128, 20)

  def forward(self, x):
      x = self.emb(x)
      x, _ = self.lstm(x)
      x = self.ln1(x)
      return x

model = myModel()

for i in train_dataloader:
  print(i[0].shape)
  print(model(i[0]).shape)
  break

torch.Size([50, 50])
torch.Size([50, 50, 20])


# LSTM을 bi-directional하게 만들기

In [5]:
from torch import nn

class myModel(nn.Module):
  def __init__(self) -> None:
      super().__init__()
      self.emb = nn.Embedding(50000, 128)
      self.lstm = nn.LSTM(128, 128, bidirectional=True)
      self.ln1 = nn.Linear(256, 20)

  def forward(self, x):
      x = self.emb(x)
      x, _ = self.lstm(x)
      x = self.ln1(x)
      return x

model = myModel()

for i in train_dataloader:
  print(i[0].shape)
  print(model(i[0]).shape)
  break

torch.Size([50, 50])
torch.Size([50, 50, 20])


# Seq2seq는 encoder model과 decoder model로 구성

In [6]:
from torch import nn

class myModel(nn.Module):
  def __init__(self) -> None:
      super().__init__()
      self.en_emb = nn.Embedding(50000, 128)
      self.en_lstm = nn.LSTM(128, 128, batch_first=True)

      self.de_emb = nn.Embedding(50000, 128)
      self.de_lstm = nn.LSTM(128, 128, batch_first=True)
      self.de_ln1 = nn.Linear(128, 20)

  def forward(self, x):
      x = self.en_emb(x)
      x, s = self.en_lstm(x)
      x, _ = self.de_lstm(x, s)
      x = self.de_ln1(x)
      return x

model = myModel()
for i in train_dataloader:
  print(i[0].shape)
  print(model(i[0]).shape)
  break

torch.Size([50, 50])
torch.Size([50, 50, 20])


# Optimization

In [9]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

model = myModel()
model.cuda()

#학습을 위한 optimizer와 loss function 설정 (learning rate 0.0001)
optimizer = Adam(params=model.parameters(), lr=0.001)
lf = CrossEntropyLoss()


#100번의 에폭을 실행
for e in range(100):
  print("\nepoch ", e)
  epoch_loss = 0
  train_correct = 0

  #선언한 모델 오브젝트를 학습가능한 상태로 변경
  model.train()

  #모든 학습데이터에 대해서 학습
  for i in train_dataloader:
    #매 배치에 대한 gradient계산 이전에 optimizer에 저장된 이전 batch에 gradient를 삭제(초기화)
    optimizer.zero_grad()

    data = i[0].cuda()
    target = i[1].reshape(-1).cuda()
    #print(target.shape)

    #결과 도출 및 정답수 연산
    output = model(data).reshape(-1,20)
    pred_label = torch.argmax(output, dim=-1)
    train_correct += sum(pred_label == target)

    #loss연산
    loss = lf(output, target.long())

    #loss backpropagation
    loss.backward()

    #gradient update
    optimizer.step()
    epoch_loss += loss.item()

  print("train loss", epoch_loss/len(train_dataloader))
  print("train acc", train_correct/len(train_dataset))

  #model이 학습되지 않는 상태로 변경
  model.eval()
  test_loss = 0
  test_correct = 0

  #gradient를 계산하지 않도록 하여 cost낭비 방지
  with torch.no_grad():
    #모든 test dataset에 대해서 결과연산
    for i in test_dataloader:
      test_data = i[0].cuda()
      test_target = i[1].reshape(-1).cuda()
      #print(test_target.shape)

      # model에 입력하기
      output = model(test_data).reshape(-1,20)
      pred_label = torch.argmax(output, dim=-1)
      test_correct += sum(pred_label == test_target)

      # loss 계산
      loss = lf(output, test_target.long())
      test_loss += loss.item()


  print("test loss", test_loss/len(test_dataloader))
  print("test acc", test_correct/len(test_dataset))




epoch  0
train loss 0.7071731425821781
train acc tensor(42.6100, device='cuda:0')
test loss 0.3403932343829762
test acc tensor(46.5938, device='cuda:0')

epoch  1
train loss 0.31540982276201246
train acc tensor(46.6425, device='cuda:0')
test loss 0.29928819699720904
test acc tensor(46.5938, device='cuda:0')

epoch  2
train loss 0.26506723258644344
train acc tensor(46.6555, device='cuda:0')
test loss 0.25639824298295105
test acc tensor(46.6599, device='cuda:0')

epoch  3
train loss 0.2262372164055705
train acc tensor(46.9350, device='cuda:0')
test loss 0.23207912661812521
test acc tensor(47.0037, device='cuda:0')

epoch  4
train loss 0.19969641733914614
train acc tensor(47.2103, device='cuda:0')
test loss 0.21429807354103436
test acc tensor(47.1783, device='cuda:0')

epoch  5
train loss 0.17775826025754213
train acc tensor(47.4913, device='cuda:0')
test loss 0.20211015099828894
test acc tensor(47.4540, device='cuda:0')

epoch  6
train loss 0.1565058244392276
train acc tensor(47.8550, d